In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd
from utils import *
from sklearn.model_selection import train_test_split
from IPython.display import display, HTML

datasets = ['Korea_grip', 'Dutch_drinking_inh', 'Dutch_drinking_wm', 'Dutch_drinking_sha', 
            'Brazil_health_heart', 'Brazil_health_stroke', 'China_glucose_women2', 'China_glucose_men2', 
            'Spain_Hair', 'China_HIV']

sizes = {
    'Dutch_drinking_inh' : 12121,
    'Dutch_drinking_wm' : 12131,
    'Dutch_drinking_sha' : 12098,
    'Brazil_health_heart' : 7728,
    'Brazil_health_stroke' : 9675,
    'Korea_grip' : 1022,
    'China_glucose_women2' : 4568,
    'China_glucose_men2' : 4360,
    'Spain_Hair' : 529,
    'China_HIV' : 2410
}

results_dict = {
    'beta' : {},
    'diff' : {},
    'R'    : {},
    'mse'  : {},
    'vol'  : {},
    'inc'  : {}
}

val_data = {}
test_data = {}

for dataset in datasets:
    val_data[dataset]  = copy.deepcopy(results_dict)
    test_data[dataset] = copy.deepcopy(results_dict)

In [2]:
dir = '../data/regr/results/diff_core'
for filename in os.listdir(dir):
    if filename[-5:] != '.json':
        continue

    expt = filename[:-5]
    attributes = expt.split(',')

    dataset = attributes[0]
    n_core = attributes[1]
    alpha = attributes[2]
    speed_type = attributes[3]
    refit = attributes[4]
    val_or_test = attributes[5]
    # print(val_or_test)
    assert val_or_test in ['val', 'test']

    expt = expt[:-len(val_or_test)]
    with open(dir + '/' + filename) as file:
        data = json.load(file)
        if val_or_test == 'val':
            # print(expt, dataset)
            val_data[dataset]['beta'][expt] = data['beta']
            val_data[dataset]['diff'][expt] = data['diff']
            val_data[dataset]['R'][expt]    = data['R']
            val_data[dataset]['mse'][expt]  = data['mse']
            val_data[dataset]['vol'][expt]  = data['vol']
            val_data[dataset]['inc'][expt]  = data['inc']
            # print(dataset, val_data[dataset]['inc'].keys())

        else:
            # print(expt, dataset)
            test_data[dataset]['beta'][expt] = data['beta']
            test_data[dataset]['diff'][expt] = data['diff']
            test_data[dataset]['R'][expt]    = data['R']
            test_data[dataset]['mse'][expt]  = data['mse']
            test_data[dataset]['vol'][expt]  = data['vol']
            test_data[dataset]['inc'][expt]  = data['inc']

In [3]:
best_mse = {}
best_diff = {}
best_low_mse_diff = {}
best_lrmse = {}
best_lrdiff = {}

for dataset in datasets:
    print(dataset)
    best_mse[dataset]  = min(val_data[dataset]['mse'],     key = val_data[dataset]['mse'].get)
    best_diff[dataset] = max(val_data[dataset]['diff'],    key = val_data[dataset]['diff'].get)

    # Compute baseline validation MSE
    X, Y, names_covariates = load_regr_data(dataset)
    n, d = X.shape
    X = np.concatenate([X, np.ones((len(X), 1))], axis = 1) # Add bias
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=101)
    _, X_val, _, Y_val = train_test_split(X_test, Y_test, test_size=0.4, random_state=101)
    base_beta, _, _ = core_fit(X_train, Y_train)
    base_mse = np.mean((X_val @ base_beta - Y_val) ** 2)
    
    val_size = int(sizes[dataset] * 0.5 * 0.4)
    low_mse = []
    includes_geq_5percent = []

    for expt in val_data[dataset]['mse']:
        if val_data[dataset]['mse'][expt] <= 2 * base_mse:
            low_mse.append(expt)
        if val_data[dataset]['inc'][expt] >= 0.05 * val_size:
            includes_geq_5percent.append(expt)

    low_mse_models = {expt: val_data[dataset]['diff'][expt] for expt in low_mse}
    best_low_mse_diff[dataset] = min(low_mse_models, key = low_mse_models.get)

    large_regions_mse = {expt: val_data[dataset]['mse'][expt] for expt in includes_geq_5percent}
    large_regions_diff = {expt: val_data[dataset]['diff'][expt] for expt in includes_geq_5percent}
    best_lrmse[dataset] = min(large_regions_mse, key = large_regions_mse.get)
    best_lrdiff[dataset] = max(large_regions_diff, key = large_regions_diff.get)

Korea_grip
Dutch_drinking_inh
Dutch_drinking_wm
Dutch_drinking_sha
Brazil_health_heart
Brazil_health_stroke
China_glucose_women2
China_glucose_men2
Spain_Hair
China_HIV


# MSE/Inclusion results

In [4]:
# summary = pd.DataFrame()
summary_data = []

for dataset in datasets:
    X, Y, names_covariates = load_regr_data(dataset)
    n, d = X.shape
    X = np.concatenate([X, np.ones((len(X), 1))], axis = 1) # Add bias

    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=101)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.4, random_state=101)

    base_beta, _, _ = core_fit(X_train, Y_train)
    
    base_mse = np.mean((X_test @ base_beta - Y_test) ** 2)
    base_inc = len(X_test)

    mse = best_mse[dataset]
    diff = best_diff[dataset]
    lmdiff = best_low_mse_diff[dataset] # This one doesn't work well
    lrmse = best_lrmse[dataset]
    lrdiff = best_lrdiff[dataset]

    inc_summary = np.array([base_inc, test_data[dataset]["inc"][lrmse], test_data[dataset]["inc"][lrdiff]]) / base_inc
    mse_summary = np.array([base_mse, test_data[dataset]["mse"][lrmse], test_data[dataset]["mse"][lrdiff]])
    inc_summary = np.around(inc_summary, 2)
    mse_summary = np.around(mse_summary, 2)
    data = [f'{mse_summary[i]} ({inc_summary[i]})' for i in range(3)]
    # summary = pd.concat(summary, np.concatenate(inc_summary, mse_summary).around(3), axis = 0)
    # data =  np.around(np.concatenate([inc_summary, mse_summary]), 3)
    summary_data.append(data)
    # print('-----------------------------------')
    # print(f'Results for {dataset}: (num incl, mse)')
    # print('-----------------------------------')
    # print(f'Base: {base_inc}, {round(base_mse, 3)}')
    # print(f'Min. MSE: {test_data[dataset]["inc"][mse]}, {round(test_data[dataset]["mse"][mse], 3)}')
    # print(f'Max. diff.: {test_data[dataset]["inc"][diff]}, {round(test_data[dataset]["mse"][diff], 3)}')
    # print(f'Max. diff. (low MSE): {test_data[dataset]["inc"][lmdiff]}, {round(test_data[dataset]["mse"][lmdiff], 3)}')
    # print('-----------------------------------')
# summary = pd.DataFrame(data = summary_data, \
#     index = datasets, \
#     columns=['Base TI', 'MSE TI', 'Diff. TI', 'LMDiff. TI', 'Base MSE', 'MSE MSE', 'Diff. MSE', 'LMDiff. MSE'])
summary = pd.DataFrame(data = summary_data, index = datasets, columns = ['Base', 'LRMSE', 'LRDiff.'])
# print(summary.to_latex())
display(HTML(summary.to_html()))

,Base,LRMSE,LRDiff.
Korea_grip,0.77 (1.0),0.88 (0.1),0.74 (0.09)
Dutch_drinking_inh,0.63 (1.0),0.68 (0.26),0.81 (0.09)
Dutch_drinking_wm,0.74 (1.0),0.76 (0.14),0.8 (0.07)
Dutch_drinking_sha,0.62 (1.0),0.51 (0.14),0.94 (0.09)
Brazil_health_heart,0.71 (1.0),0.76 (0.07),0.76 (0.07)
Brazil_health_stroke,0.56 (1.0),0.43 (0.73),0.69 (0.04)
China_glucose_women2,0.78 (1.0),13.74 (0.04),13.74 (0.04)
China_glucose_men2,0.97 (1.0),0.73 (0.08),0.73 (0.08)
Spain_Hair,0.79 (1.0),0.68 (0.1),0.68 (0.1)
China_HIV,0.83 (1.0),0.9 (0.39),0.69 (0.23)


# Coefficient/Region results

In [5]:
import statsmodels.api as sm

results = {}

for dataset in datasets:
    results[dataset] = {}

    mse = best_mse[dataset]
    diff = best_diff[dataset]
    lmdiff = best_low_mse_diff[dataset]
    lrmse = best_lrmse[dataset]
    lrdiff = best_lrdiff[dataset]

    X, Y, names_covariates = load_regr_data(dataset)
    B = np.column_stack([np.min(X, axis = 0) - 0.1, np.max(X, axis = 0) + 0.1])

    names_covariates = list(names_covariates)
    names_covariates.insert(0, 'Bias')
    X = sm.add_constant(X)
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=101)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.4, random_state=101)

    base_R   = B
    mse_R    = np.array(val_data[dataset]['R'][mse])
    diff_R   = np.array(val_data[dataset]['R'][diff])
    lmdiff_R = np.array(val_data[dataset]['R'][lmdiff])
    lrmse_R    = np.array(val_data[dataset]['R'][lrmse])
    lrdiff_R   = np.array(val_data[dataset]['R'][lrdiff])

    mse_ind    = in_box(X_train[:, 1:], mse_R)
    diff_ind   = in_box(X_train[:, 1:], diff_R)
    lmdiff_ind = in_box(X_train[:, 1:], lmdiff_R)
    lrmse_ind  = in_box(X_train[:, 1:], lrmse_R)
    lrdiff_ind = in_box(X_train[:, 1:], lrdiff_R)
    
    base_reg = sm.OLS(Y_train, X_train).fit()

    # X_mse = X_train[mse_ind]
    # Y_mse = Y_train[mse_ind]
    # mse_reg = sm.OLS(Y_mse, X_mse).fit()

    # X_diff = X_train[diff_ind]
    # Y_diff = Y_train[diff_ind]
    # diff_reg = sm.OLS(Y_diff, X_diff).fit()

    # X_lmdiff = X_train[lmdiff_ind]
    # Y_lmdiff = Y_train[lmdiff_ind]
    # lmdiff_reg = sm.OLS(Y_lmdiff, X_lmdiff).fit()

    X_lrmse = X_train[lrmse_ind]
    Y_lrmse = Y_train[lrmse_ind]
    lrmse_reg = sm.OLS(Y_lrmse, X_lrmse).fit()

    X_lrdiff = X_train[lrdiff_ind]
    Y_lrdiff = Y_train[lrdiff_ind]
    lrdiff_reg = sm.OLS(Y_lrdiff, X_lrdiff).fit()

    # results[dataset]['coeffs'] = pd.DataFrame()
    coeffs = pd.DataFrame()
    region = pd.DataFrame()
    prec = 2
    for i, var in enumerate(names_covariates):
        coeffs[var] = [f'{round(base_reg.params[i], prec)}  ({round(base_reg.pvalues[i], prec)})', 
                       f'{round(lrmse_reg.params[i], prec)}  ({round(lrmse_reg.pvalues[i], prec)})',
                       f'{round(lrdiff_reg.params[i], prec)}  ({round(lrdiff_reg.pvalues[i], prec)})']
        if i > 0:
            region[var] = [(round(base_R[i - 1, 0], 3), round(base_R[i - 1, 1], 3)),
                           (round(lrmse_R[i - 1, 0], 3), round(lrmse_R[i - 1, 1], 3)),
                           (round(lrdiff_R[i - 1, 0], 3), round(lrdiff_R[i - 1, 1], 3))]

    coeffs.index = ['Base', 'LRMSE', 'LRDiff.']
    region.index = ['Base', 'LRMSE', 'LRDiff.']
    results[dataset]['coeffs'] = coeffs
    results[dataset]['R'] = region

    # print('----------------------------------------------------')
    # print(f'Data: {dataset}')
    # print('----------------------------------------------------')
    # print('Coeffs (weight, pval):')
    # print(coeffs)
    # print('----------------------------------------------------')
    # print('Region:')
    # print(region)
    # print('----------------------------------------------------')
    print('----------------------------------------------------')
    print(f'Data: {dataset}')
    print('----------------------------------------------------')
    # print(coeffs.T.to_latex())
    # print(region.T.to_latex())
    display(HTML(coeffs.T.to_html()))
    display(HTML(region.T.to_html()))
    print('----------------------------------------------------')

----------------------------------------------------
Data: Korea_grip
----------------------------------------------------


,Base,LRMSE,LRDiff.
Bias,0.05 (0.12),0.08 (0.72),0.1 (0.76)
total_s_hand,-0.03 (0.8),0.12 (0.87),-0.31 (0.69)
JSN_hand,-0.14 (0.15),-0.18 (0.7),0.22 (0.66)
OP_hand,0.02 (0.79),0.07 (0.87),0.22 (0.57)
total_s_knee,0.08 (0.75),-1.17 (0.51),-1.71 (0.17)
OP_knee,-0.0 (0.99),0.61 (0.51),0.95 (0.18)
JSN_knee,-0.09 (0.42),0.52 (0.47),0.81 (0.18)
age,-0.39 (0.0),-0.45 (0.06),-0.53 (0.1)
BMI,0.14 (0.0),0.3 (0.17),0.27 (0.32)
smoking_c,-0.03 (0.48),0.06 (0.66),0.09 (0.52)


,Base,LRMSE,LRDiff.
total_s_hand,"(-2.796, 6.123)","(-2.796, 6.123)","(-2.796, 6.123)"
JSN_hand,"(-2.747, 3.533)","(-2.747, 0.731)","(-2.747, 0.731)"
OP_hand,"(-2.877, 5.334)","(-2.877, 1.464)","(-0.421, 5.334)"
total_s_knee,"(-2.456, 4.987)","(-2.456, 4.987)","(-2.456, 4.987)"
OP_knee,"(-2.474, 5.632)","(-2.474, -0.068)","(-2.474, 5.632)"
JSN_knee,"(-2.463, 2.673)","(-2.463, 0.105)","(-2.363, -0.306)"
age,"(-1.898, 3.61)","(-1.898, 0.266)","(-0.68, 0.93)"
BMI,"(-3.452, 4.225)","(-0.441, 4.225)","(-3.452, 0.386)"
smoking_c,"(-0.689, 1.799)","(-0.689, 1.699)","(-0.689, 1.699)"
alcohol_c,"(-1.545, 0.792)","(-1.445, 0.792)","(-1.445, 0.792)"


----------------------------------------------------
----------------------------------------------------
Data: Dutch_drinking_inh
----------------------------------------------------


,Base,LRMSE,LRDiff.
Bias,0.02 (0.58),0.11 (0.18),-0.51 (0.12)
Imputation__1.0,-0.03 (0.52),-0.11 (0.16),-0.08 (0.61)
Imputation__2.0,-0.02 (0.73),-0.03 (0.73),-0.08 (0.64)
Imputation__3.0,-0.03 (0.46),-0.1 (0.24),-0.12 (0.46)
Imputation__4.0,-0.01 (0.76),-0.04 (0.59),-0.04 (0.83)
Imputation__5.0,-0.03 (0.56),-0.06 (0.47),-0.07 (0.67)
t1inh,0.57 (0.0),0.51 (0.0),0.47 (0.0)
sex,0.07 (0.0),0.09 (0.0),0.08 (0.04)
t1age,-0.02 (0.15),-0.02 (0.25),0.03 (0.7)
t1ses,0.08 (0.0),0.12 (0.0),0.06 (0.46)


...

,Base,LRMSE,LRDiff.
Age,"(-1.377, 3.615)","(-1.085, -0.957)","(-1.085, -0.957)"
Education,"(-1.762, 0.912)","(-0.012, 0.912)","(-0.012, 0.912)"
EmploymentS,"(-1.311, 2.333)","(-1.311, 1.085)","(-1.311, 1.085)"
HairDye,"(-0.531, 2.418)","(-0.531, 2.418)","(-0.531, 2.318)"
PhysicalAct,"(-1.022, 1.185)","(-0.922, 1.185)","(-0.922, 1.185)"


----------------------------------------------------
----------------------------------------------------
Data: China_HIV
----------------------------------------------------


,Base,LRMSE,LRDiff.
Bias,-0.01 (0.93),0.03 (0.88),-0.71 (0.0)
route_2.0,-0.08 (0.36),-0.07 (0.66),0.09 (0.59)
route_3.0,0.08 (0.36),0.01 (0.95),0.06 (0.68)
route_4.0,0.12 (0.65),-0.25 (0.53),-2.82 (0.0)
route_5.0,0.16 (0.15),-0.0 (1.0),0.07 (0.72)
route_6.0,0.34 (0.25),0.34 (0.36),1.65 (0.04)
route_7.0,0.02 (0.81),0.1 (0.48),0.05 (0.76)
sex_2,-0.13 (0.14),-0.14 (0.28),0.24 (0.15)
ethni_2.0,-0.01 (0.86),-0.01 (0.93),0.23 (0.04)
ethni_3.0,0.34 (0.02),0.28 (0.16),0.94 (0.02)


,Base,LRMSE,LRDiff.
route_2.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.0)"
route_3.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.0)"
route_4.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.1)"
route_5.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.0)"
route_6.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.1)"
route_7.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.1)"
sex_2,"(-0.1, 1.1)","(-0.1, 1.0)","(-0.1, 1.0)"
ethni_2.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.1)"
ethni_3.0,"(-0.1, 1.1)","(-0.1, 1.1)","(-0.1, 1.0)"
relig_2.0,"(-0.1, 1.1)","(-0.1, 1.0)","(-0.1, 1.1)"


----------------------------------------------------
